In [2]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

In [32]:
from utils import get_nse_payload, this_is_nse_index
import pandas as pd
import yaml

In [33]:
def make_nse_option_url(symbol) -> str:

    if this_is_nse_index(symbol):
        url = 'https://www.nseindia.com/api/option-chain-indices?symbol=' + symbol.upper()
    else:
        url = 'https://www.nseindia.com/api/option-chain-equities?symbol=' + symbol.upper()

    return url

In [34]:
def nse_web_json_to_df(symbol: str) -> pd.DataFrame:
    """Makes option df of nse symbol from nseindia.com"""

    url = make_nse_option_url(symbol)

    response = get_nse_payload(url)

    js = response.json()
    data = js.get('records').get('data')

    matches = ['CE', 'PE']

    # Extract options dictionary
    options = dict()

    for d in data:
        for k, v in d.items():
            if any(x in k for x in matches):
                v['right'] = k
                s = v['identifier']
                options[s] = v

    df_opts = pd.DataFrame.from_dict(options, orient='index')

    return df_opts


In [51]:
def clean_nse_web_options(df_opts: pd.DataFrame) -> pd.DataFrame:
    """Cleans nse web option df with standardized column names from /data/master/column_maps.yml"""

    with open(ROOT / 'data' / 'master'/ 'column_maps.yml', 'rb') as f:
        data_map = yaml.safe_load(f)

    map_cols = data_map['NSE_WEB_OPTIONS']

    # reverse to keep structure of cols in df
    col_map = {v:k for k, v in map_cols.items()}

    # remove unncessary columns
    df_opts = df_opts.rename(columns=col_map)
    df_opts = df_opts[list(col_map.values())].\
        sort_values(['expiry', 'strike', 'right'], 
                    ascending=[True, False, False])
    df_opts = df_opts.reset_index(drop=True)\
                        .drop(columns=['id'])
    
    df_opts.expiry = pd.to_datetime(df_opts.expiry, format='%d-%b-%Y')
    
    return df_opts

In [43]:
def get_an_nse_option(symbol: str) -> pd.DataFrame:

    df = nse_web_json_to_df(symbol)\
            .pipe(clean_nse_web_options)
    
    return df


In [52]:
# Input
symbol = 'BANKNIFTY'
# nse_web_json_to_df(symbol)
df_opts = get_an_nse_option(symbol=symbol)


In [53]:
df_opts

,symbol,undPrice,strike,expiry,right,oi,oi_chg,pchg_oi,volume,change,p_change,total_buy_qty,total_sell_qty,bid_qty,bid_price,ask_qty,ask_price,iv,price
0,BANKNIFTY,49396.75,57000,2024-05-08,PE,0,0,0.000000,0,0.00,0.000000,1185,1185,885,6938.75,300,8025.55,0.00,0.00
1,BANKNIFTY,49396.75,57000,2024-05-08,CE,7266,7266,0.000000,16819,1.05,80.769231,160365,50400,3255,2.20,1065,2.35,34.50,2.35
2,BANKNIFTY,49396.75,56500,2024-05-08,PE,0,0,0.000000,0,0.00,0.000000,1185,1185,300,6459.85,300,7517.95,0.00,0.00
3,BANKNIFTY,49396.75,56500,2024-05-08,CE,11565,8983,347.908598,22456,-0.60,-20.689655,129285,49275,525,2.30,300,2.60,32.49,2.30
4,BANKNIFTY,49396.75,56000,2024-05-08,PE,0,0,0.000000,0,0.00,0.000000,1185,1185,300,5989.75,300,6936.30,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,BANKNIFTY,49396.75,42000,2024-07-31,PE,0,0,0.000000,0,0.00,0.000000,900,0,900,10.15,0,0.00,0.00,0.00
1442,BANKNIFTY,49396.75,42000,2024-07-31,CE,0,0,0.000000,0,0.00,0.000000,0,0,0,0.00,0,0.00,0.00,0.00
1443,BANKNIFTY,49396.75,41500,2024-07-31,PE,0,0,0.000000,0,0.00,0.000000,900,0,900,6.15,0,0.00,0.00,0.00
1444,BANKNIFTY,49396.75,41000,2024-07-31,PE,0,0,0.000000,0,0.00,0.000000,1065,0,15,23.75,0,0.00,0.00,0.00


# Options to df